In [6]:
import sqlite3
import pandas as pd

# Kết nối hoặc tạo cơ sở dữ liệu SQLite
conn = sqlite3.connect("students.db")
cursor = conn.cursor()

# Tạo bảng student
cursor.execute('''CREATE TABLE IF NOT EXISTS student (
    student_id INTEGER PRIMARY KEY,
    name TEXT,
    class TEXT,
    course_id INTEGER,
    score REAL
)''')

# Tạo bảng course
cursor.execute('''CREATE TABLE IF NOT EXISTS course (
    id INTEGER PRIMARY KEY,
    course_name TEXT
)''')

# Thêm dữ liệu vào bảng student
students_data = [
    (1, "Nguyen Minh Hoang", "May Tinh", 12, 6.7),
    (2, "Tran Thi Lan", "Kinh Te", 34, 9.2),
    (3, "Pham Van Nam", "Toan Tin", None, 7.9),
    (4, "Le Thanh Huyen", "Toan Tin", 20, 7.2),
    (5, "Vu Quoc Anh", "May Tinh", 24, 8.0),
    (6, "Dang Thuy Linh", "May Tinh", 24, 5.5),
    (7, "Bui Tien Dung", "Kinh Te", 34, 9.2),
    (8, "Ho Ngoc Mai", "Toan Tin", 20, 8.8),
    (9, "Duong Huu Phuc", "Kinh Te", None, 7.2),
    (10, "Cao Thi Hanh", "May Tinh", None, 7.0)
]
cursor.executemany("INSERT OR IGNORE INTO student VALUES (?, ?, ?, ?, ?)", students_data)

# Thêm dữ liệu vào bảng course
courses_data = [
    (12, "Giai tich"),
    (34, "Thong ke"),
    (26, "Tin hoc")
]
cursor.executemany("INSERT OR IGNORE INTO course VALUES (?, ?)", courses_data)

conn.commit()

print("Dữ liệu trong bảng student:")
cursor.execute("SELECT * FROM student")
students = cursor.fetchall()
df_students = pd.DataFrame(students, columns=[x[0] for x in cursor.description])
print(df_students)

print("\nDữ liệu trong bảng course:")
cursor.execute("SELECT * FROM course")
courses = cursor.fetchall()
df_courses = pd.DataFrame(courses, columns=[x[0] for x in cursor.description])
print(df_courses)


Dữ liệu trong bảng student:
   student_id               name     class  course_id  score
0           1  Nguyen Minh Hoang  May Tinh         12    6.7
1           2       Tran Thi Lan   Kinh Te         34    9.2
2           3       Pham Van Nam  Toan Tin         12    7.9
3           4     Le Thanh Huyen  Toan Tin         20    7.2
4           5        Vu Quoc Anh  May Tinh         24    8.0
5           6     Dang Thuy Linh  May Tinh         24    5.5
6           7      Bui Tien Dung   Kinh Te         34    9.2
7           8        Ho Ngoc Mai  Toan Tin         20    8.8
8           9     Duong Huu Phuc   Kinh Te         12    7.2
9          10       Cao Thi Hanh  May Tinh         12    7.0

Dữ liệu trong bảng course:
   id course_name
0  12   Giai tich
1  26     Tin hoc
2  34    Thong ke


In [2]:
# INNER JOIN
query_inner = """
SELECT student.*, course.course_name
FROM student
INNER JOIN course ON student.course_id = course.id
"""
df_inner = pd.read_sql_query(query_inner, conn)
print("\nINNER JOIN:")
print(df_inner)

# LEFT JOIN
query_left = """
SELECT student.*, course.course_name
FROM student
LEFT JOIN course ON student.course_id = course.id
"""
df_left = pd.read_sql_query(query_left, conn)
print("\nLEFT JOIN:")
print(df_left)


INNER JOIN:
   student_id               name     class  course_id  score course_name
0           1  Nguyen Minh Hoang  May Tinh         12    6.7   Giai tich
1           2       Tran Thi Lan   Kinh Te         34    9.2    Thong ke
2           7      Bui Tien Dung   Kinh Te         34    9.2    Thong ke

LEFT JOIN:
   student_id               name     class  course_id  score course_name
0           1  Nguyen Minh Hoang  May Tinh       12.0    6.7   Giai tich
1           2       Tran Thi Lan   Kinh Te       34.0    9.2    Thong ke
2           3       Pham Van Nam  Toan Tin        NaN    7.9        None
3           4     Le Thanh Huyen  Toan Tin       20.0    7.2        None
4           5        Vu Quoc Anh  May Tinh       24.0    8.0        None
5           6     Dang Thuy Linh  May Tinh       24.0    5.5        None
6           7      Bui Tien Dung   Kinh Te       34.0    9.2    Thong ke
7           8        Ho Ngoc Mai  Toan Tin       20.0    8.8        None
8           9     Duong Hu

In [3]:
# RIGHT JOIN (Mô phỏng bằng cách đổi vị trí bảng và dùng LEFT JOIN)
query_right = """
SELECT course.*, student.*
FROM course
LEFT JOIN student ON student.course_id = course.id
"""
df_right = pd.read_sql_query(query_right, conn)
print("\nRIGHT JOIN:")
print(df_right)

# FULL OUTER JOIN (Mô phỏng bằng UNION của LEFT JOIN và RIGHT JOIN)
query_full_outer = """
SELECT student.*, course.course_name
FROM student
LEFT JOIN course ON student.course_id = course.id
UNION
SELECT student.*, course.course_name
FROM course
LEFT JOIN student ON student.course_id = course.id;
"""
df_full_outer = pd.read_sql_query(query_full_outer, conn)
print("\nFULL OUTER JOIN:")
print(df_full_outer)


RIGHT JOIN:
   id course_name  student_id               name     class  course_id  score
0  12   Giai tich         1.0  Nguyen Minh Hoang  May Tinh       12.0    6.7
1  26     Tin hoc         NaN               None      None        NaN    NaN
2  34    Thong ke         7.0      Bui Tien Dung   Kinh Te       34.0    9.2
3  34    Thong ke         2.0       Tran Thi Lan   Kinh Te       34.0    9.2

FULL OUTER JOIN:
    student_id               name     class  course_id  score course_name
0          NaN               None      None        NaN    NaN     Tin hoc
1          1.0  Nguyen Minh Hoang  May Tinh       12.0    6.7   Giai tich
2          2.0       Tran Thi Lan   Kinh Te       34.0    9.2    Thong ke
3          3.0       Pham Van Nam  Toan Tin        NaN    7.9        None
4          4.0     Le Thanh Huyen  Toan Tin       20.0    7.2        None
5          5.0        Vu Quoc Anh  May Tinh       24.0    8.0        None
6          6.0     Dang Thuy Linh  May Tinh       24.0    5.5     

In [4]:
# Cập nhật course_id còn thiếu với một giá trị hợp lệ từ bảng course
cursor.execute("""
UPDATE student
SET course_id = (SELECT id FROM course ORDER BY RANDOM() LIMIT 1)
WHERE course_id IS NULL;
""")

# Xóa các sinh viên có course_id không tồn tại trong bảng course
cursor.execute("""
DELETE FROM student
WHERE course_id NOT IN (SELECT id FROM course);
""")

conn.commit()

# Tính tổng số sinh viên, điểm trung bình theo từng lớp
query_class_avg = """
SELECT class, COUNT(*) AS total_students, AVG(score) AS avg_score
FROM student
GROUP BY class;
"""
df_class_avg = pd.read_sql_query(query_class_avg, conn)
print("Tổng số sinh viên và điểm trung bình theo từng lớp:")
print(df_class_avg)

# Tính tổng số sinh viên, điểm trung bình theo từng môn học
query_course_avg = """
SELECT course.course_name, COUNT(student.student_id) AS total_students, AVG(student.score) AS avg_score
FROM student
JOIN course ON student.course_id = course.id
GROUP BY course.course_name;
"""
df_course_avg = pd.read_sql_query(query_course_avg, conn)
print("Tổng số sinh viên và điểm trung bình theo từng môn học:")
print(df_course_avg)

# Phân loại thi đua theo số điểm
query_ranking = """
SELECT name, class, course_id, score,
    CASE
        WHEN score >= 9.0 THEN 'Xuất sắc'
        WHEN score >= 6.0 THEN 'Tốt'
        ELSE 'Kém'
    END AS ranking
FROM student;
"""
df_ranking = pd.read_sql_query(query_ranking, conn)
print("Phân loại thi đua theo điểm:")
print(df_ranking)

Tổng số sinh viên và điểm trung bình theo từng lớp:
      class  total_students  avg_score
0   Kinh Te               3   8.533333
1  May Tinh               2   6.850000
2  Toan Tin               1   7.900000
Tổng số sinh viên và điểm trung bình theo từng môn học:
  course_name  total_students  avg_score
0   Giai tich               4        7.2
1    Thong ke               2        9.2
Phân loại thi đua theo điểm:
                name     class  course_id  score   ranking
0  Nguyen Minh Hoang  May Tinh         12    6.7       Tốt
1       Tran Thi Lan   Kinh Te         34    9.2  Xuất sắc
2       Pham Van Nam  Toan Tin         12    7.9       Tốt
3      Bui Tien Dung   Kinh Te         34    9.2  Xuất sắc
4     Duong Huu Phuc   Kinh Te         12    7.2       Tốt
5       Cao Thi Hanh  May Tinh         12    7.0       Tốt


In [5]:
# Đọc dữ liệu từ bảng student
query_student = "SELECT name, class, course_id, score FROM student;"
df_student = pd.read_sql_query(query_student, conn)

# Xếp hạng theo điểm số chung
df_student['rank_overall'] = df_student['score'].rank(method='dense', ascending=False)

# Xếp hạng theo lớp
df_student['rank_class'] = df_student.groupby('class')['score'].rank(method='dense', ascending=False)

# Xếp hạng theo môn học
df_student['rank_course'] = df_student.groupby('course_id')['score'].rank(method='dense', ascending=False)

print("Xếp hạng sinh viên theo điểm số chung:")
print(df_student[['name', 'class', 'course_id', 'score', 'rank_overall']])

print("Xếp hạng sinh viên theo lớp học:")
print(df_student[['name', 'class', 'course_id', 'score', 'rank_class']])

print("Xếp hạng sinh viên theo môn học:")
print(df_student[['name', 'class', 'course_id', 'score', 'rank_course']])

# Top 3 sinh viên cao nhất và thấp nhất
print("Top 3 sinh viên có điểm cao nhất:")
print(df_student.nsmallest(3, 'rank_overall'))

print("Top 3 sinh viên có điểm thấp nhất:")
print(df_student.nlargest(3, 'rank_overall'))


Xếp hạng sinh viên theo điểm số chung:
                name     class  course_id  score  rank_overall
0  Nguyen Minh Hoang  May Tinh         12    6.7           5.0
1       Tran Thi Lan   Kinh Te         34    9.2           1.0
2       Pham Van Nam  Toan Tin         12    7.9           2.0
3      Bui Tien Dung   Kinh Te         34    9.2           1.0
4     Duong Huu Phuc   Kinh Te         12    7.2           3.0
5       Cao Thi Hanh  May Tinh         12    7.0           4.0
Xếp hạng sinh viên theo lớp học:
                name     class  course_id  score  rank_class
0  Nguyen Minh Hoang  May Tinh         12    6.7         2.0
1       Tran Thi Lan   Kinh Te         34    9.2         1.0
2       Pham Van Nam  Toan Tin         12    7.9         1.0
3      Bui Tien Dung   Kinh Te         34    9.2         1.0
4     Duong Huu Phuc   Kinh Te         12    7.2         2.0
5       Cao Thi Hanh  May Tinh         12    7.0         1.0
Xếp hạng sinh viên theo môn học:
                name     cl

In [6]:
import sqlite3
import pandas as pd
from datetime import datetime, timedelta

# Kết nối hoặc tạo cơ sở dữ liệu SQLite
conn = sqlite3.connect("students.db")
cursor = conn.cursor()

# Đọc dữ liệu từ bảng student
query_student = "SELECT student_id, name, class, course_id, score FROM student;"
df_student = pd.read_sql_query(query_student, conn)

# Xếp hạng theo điểm số chung
df_student['rank_overall'] = df_student['score'].rank(method='dense', ascending=False)

# Tính ngày tốt nghiệp bằng cách cộng thứ hạng vào thời gian hiện tại
df_student['graduation_date'] = [datetime.now() + timedelta(days=int(rank)) for rank in df_student['rank_overall']]

# Cập nhật graduation_date vào database
for index, row in df_student.iterrows():
    cursor.execute("""
    UPDATE student
    SET graduation_date = ?
    WHERE student_id = ?;
    """, (row['graduation_date'].strftime('%Y-%m-%d %H:%M:%S'), row['student_id']))  

conn.commit()

print("\nXếp hạng sinh viên theo điểm số chung:")
print(df_student[['name', 'class', 'course_id', 'score', 'rank_overall', 'graduation_date']])




Xếp hạng sinh viên theo điểm số chung:
                name     class  course_id  score  rank_overall  \
0  Nguyen Minh Hoang  May Tinh         12    6.7           5.0   
1       Tran Thi Lan   Kinh Te         34    9.2           1.0   
2       Pham Van Nam  Toan Tin         12    7.9           2.0   
3      Bui Tien Dung   Kinh Te         34    9.2           1.0   
4     Duong Huu Phuc   Kinh Te         12    7.2           3.0   
5       Cao Thi Hanh  May Tinh         12    7.0           4.0   

             graduation_date  
0 2025-03-17 00:37:07.305284  
1 2025-03-13 00:37:07.305284  
2 2025-03-14 00:37:07.305284  
3 2025-03-13 00:37:07.305284  
4 2025-03-15 00:37:07.305284  
5 2025-03-16 00:37:07.305284  
